In [1]:
import pandas as pd
import numpy as np
import choix
from sklearn.preprocessing import normalize
# get data through terminal: wget https://raw.githubusercontent.com/kpelechrinis/BT-model/master/nba1617.csv

In [3]:
# An example of Bradley-Terry model with win-loss relationships in the NBA

df = pd.read_csv("nba1617.csv") # : a file with the game results from the 2016-17 NBA season
teams = list(set(df.Home.unique()))
t = list(np.sort(teams))

data = []

for i in range(len(df.Visitor)):
    if df.PTSV[i] > df.PTSH[i]: # if visitor win
        el = (t.index(df.Visitor[i]),t.index(df.Home[i])) # set vistor first in tuple 
    else:
        el = (t.index(df.Home[i]),t.index(df.Visitor[i])) # else, set home first in tuple
    data.append(el)

skills = choix.ilsr_pairwise(30, data) # pairwise comparison

# ------------------------------

prob_mat = np.zeros([30]) # create first empty array

for i in range(30):
    win_probs = []
    for j in range(30):
        win_probs.append(choix.probabilities([i, j], skills)[0])
        
    if prob_mat.sum() == 0:
        prob_mat = prob_mat + win_probs #first iteration, replace zero-array
        
    else:
        prob_mat = np.vstack((prob_mat,np.array(win_probs))) #concurrent iteration, append array as row-vector 

# Out puts a probability matrix that captures the win probability of the row team winning the column team

# OLD:
# for i in range(30):
#     win_probs = []
#     for j in range(30):
#         win_probs.append(choix.probabilities([i, j], skills)[0])
#     print(win_probs) # a probability matrix that captures the win probability of the row team winning the column team

In [4]:
skills_norm = (skills - skills.min())/(skills.max()-skills.min())# fit between o and 1
ranking = pd.DataFrame(list(zip(t,list(skills_norm))), columns = ['team','score']).sort_values('score', ascending=False)
ranking
# seem reasnoble according to https://www.espn.com/nba/standings/_/season/2017
# not sure why I want to nor it.. Seems prudent though

,team,score
9,Golden State Warriors,1.000000
26,San Antonio Spurs,0.844208
10,Houston Rockets,0.715555
1,Boston Celtics,0.647564
12,Los Angeles Clippers,0.636289
28,Utah Jazz,0.630680
5,Cleveland Cavaliers,0.614010
27,Toronto Raptors,0.604052
29,Washington Wizards,0.566760
20,Oklahoma City Thunder,0.560534
